In [1]:
%pwd

'c:\\Users\\tikad\\Desktop\\medical Chatbot\\End-to-end-Medical-Chatbot-Generative-AI-main\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\tikad\\Desktop\\medical Chatbot\\End-to-end-Medical-Chatbot-Generative-AI-main'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
extracted_data

[Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION'),
 Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 2}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n1\nA-B\nJACQUELINE L. LONGE, PROJECT EDITOR\n'),
 Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 3}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n2\nC-F\nJACQUELINE L. LONGE, PROJECT EDITOR\n'),
 Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 4}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n3\nG-M\nJACQUELINE L. LONGE, PROJECT EDITOR\n'),
 Document(metadata={'source': 'Data\\Medical_book.pdf', 'page': 5}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION\nVOLUME\n\x81\n4\nN-S\nJACQUELINE L. LONGE, PROJECT ED

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 39994


In [10]:
# text_chunks

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings


In [15]:
embeddings = download_hugging_face_embeddings()

c:\Users\tikad\Desktop\medical Chatbot\End-to-end-Medical-Chatbot-Generative-AI-main\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [22]:
from pinecone import Pinecone, ServerlessSpec
import os

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_6Ah8nH_JsPqoT59nZma6BnSXmintSQkLTC5b7nVbcEMhbrF3j5J79sxboS7Nh8uENX5GK8")

index_name = "medicalbot"

# Create index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [23]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_6Ah8nH_JsPqoT59nZma6BnSXmintSQkLTC5b7nVbcEMhbrF3j5J79sxboS7Nh8uENX5GK8"
#os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [25]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
docsearch

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("What is Fever?")

In [29]:
retrieved_docs

[Document(id='b0a0211a-2b14-419c-a206-db7af822eb8a', metadata={'page': 1492.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Description\nFever is a natural response of the body that helps\nin fighting off foreign substances, such as microorgan-\nisms, toxins, etc. Body temperature is set by the\nthermoregulatory center, located in an area in the\nbrain called hypothalamus. Body temperature is not\nconstant all day, but actually is lowest at 6 A.M. and\nhighest around 4–6 P.M. In addition, temperature\nvaries in different regions of the body; for example,\nrectal and urine temperatures are about one degree'),
 Document(id='23bc45b5-e3a2-46f0-9347-7fea2c467280', metadata={'page': 1793.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Hemorrhagic fevers'),
 Document(id='b030a9d6-7b19-4b91-a311-9fcbc1498d22', metadata={'page': 1489.0, 'source': 'Data\\Medical_book.pdf'}, page_content='fever that lasts for several weeks is associated with\nautoimmune diseases such as lupus or with

In [37]:
import os
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key="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"
)

def get_groq_response(prompt, temperature=0.4, max_tokens=500):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mixtral-8x7b-32768",  
        temperature=temperature,
        max_tokens=max_tokens
    )
    return chat_completion.choices[0].message.content

# Example usage
prompt = "what is Acne"
response = get_groq_response(prompt)
print(response)

Acne is a common skin condition that affects people of all ages, but it is particularly prevalent among teenagers. It occurs when the hair follicles become plugged with oil and dead skin cells, leading to whiteheads, blackheads, or pimples. Acne can appear anywhere on the body, but it is most commonly found on the face, chest, and back.

Acne is caused by several factors, including:

1. Excess oil production: The sebaceous glands produce an oily substance called sebum, which helps keep the skin moisturized. However, when the glands produce too much sebum, it can mix with dead skin cells and clog the pores, leading to acne.
2. Hair follicle irritation: When the hair follicles become irritated, they can become inflamed and lead to the formation of pimples.
3. Bacteria: The bacteria Propionibacterium acnes (P. acnes) naturally live on the skin and can contribute to the development of acne. When P. acnes bacteria become trapped in the pores, they can cause inflammation and infection.
4. Ho

In [1]:
prompt = "what is Acne"
response = get_groq_response(prompt)
print(response)

NameError: name 'get_groq_response' is not defined

In [31]:
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize Groq LLM
llm = ChatGroq(
    api_key="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",
    model_name="mixtral-8x7b-32768"
)

# Your system prompt remains the same
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [40]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_6Ah8nH_JsPqoT59nZma6BnSXmintSQkLTC5b7nVbcEMhbrF3j5J79sxboS7Nh8uENX5GK8")
index_name = "medicalbot"

# Initialize the embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# Get the index
index = pc.Index(index_name)

# Initialize Pinecone vector store
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text"  # The key that contains the text in your Pinecone vectors
)

# Create retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}  # Return top 3 most relevant documents
)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input": "what is Acne"})
print(response["answer"])

Acne is a common skin disease that results from clogged pores, leading to pimples on the face, chest, and back. Different types of anti-acne drugs are used for various purposes, from mild to severe acne. For instance, benzoyl peroxide or tretinoin can treat mild to moderately severe acne, while isotretinoin (Accutane) is prescribed for severe, disfiguring acne.
